In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler

In [4]:
full_op = pd.read_csv('op_for_model.csv')

In [6]:
full_op.drop(['Unnamed: 0', 'Fare'], axis=1, inplace=True)

In [7]:
full_op.head()

,Age,Parch,Pclass,SibSp,Survived,Embarked_Dummy,title,sex,total_family,Fare_bin_id,Pclass_Fare_Category2,SurnameNum,Fare_new
0,0,0,3,1,0,0,0,1,1,0,5,2,1.981001
1,0,0,1,1,1,1,1,0,1,1,1,2,4.266662
2,0,0,3,0,1,0,2,0,0,2,5,1,2.070022
3,0,0,1,1,1,0,1,0,1,1,1,2,3.972177
4,0,0,3,0,0,0,0,1,0,2,5,2,2.085672


In [8]:
train_set = full_op[:891]
test_set = full_op[891:]

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

In [10]:
RF_para_grid = {'n_estimators' : [200, 300, 400,500,700,800,1100,1200], 
                'criterion': ['entropy'],
                'max_depth' : [6, 8, 9,11,12,13],
                'max_features': ['auto',5,7,9,10,11]}

In [12]:
X_train = train_set.drop('Survived', axis = 1)
y_train = train_set['Survived']
X_test = test_set.drop('Survived', axis=1)

In [27]:
X_train.loc[X_train['Fare_new'] == -np.inf] = 0

In [32]:
X_test.loc[X_test['Fare_new'] == -np.inf] = 0

In [18]:
y_train.isnull().any()

False

In [29]:
RFC = RandomForestClassifier(random_state=42)
RF_Grid = GridSearchCV(RFC, RF_para_grid, cv=5, n_jobs=5, verbose=1, scoring = 'accuracy')
RF_Grid.fit(X_train, y_train)
RF_Model_Result = pd.DataFrame(RF_Grid.cv_results_)
RF_Model_Result.sort_values('rank_test_score', axis=0, ascending=True, inplace=True)
RF_Model_Result = RF_Model_Result[['params', 'rank_test_score', 'mean_test_score', 'mean_train_score']]
RF_Model_Result.to_csv('RF_MODEL_RESULT3.csv')
joblib.dump(RF_Grid.best_estimator_, 'RF_BEST_MODEL3.pkl')

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   11.7s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:   52.1s
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:  2.0min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed:  3.6min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed:  6.0min
[Parallel(n_jobs=5)]: Done 1440 out of 1440 | elapsed:  7.0min finished
C:\Users\naive\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\naive\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training sco

['RF_BEST_MODEL3.pkl']

In [30]:
df_test = pd.read_csv('E:/Kaggle/Titanic/titanic_data/test.csv')

In [33]:
RF_predict = RF_Grid.best_estimator_.predict(X_test)

In [34]:
submission_RF = pd.DataFrame({'PassengerId': df_test['PassengerId'],
                              'Survived' : pd.Series(RF_predict)
})

In [35]:
submission_RF.to_csv('submission_RF2.csv')